In [ ]:
!pip install langchain requests numpy pydantic

In [ ]:
import os
from typing import List, Dict, Any, Tuple
from datetime import datetime, timedelta
import json
from dataclasses import dataclass
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.schema import BaseMessage
import requests
from pydantic import BaseModel
import numpy as np

In [ ]:
class DeepSeekLLM(LLM):
    api_key: str = Field(..., description="Your DeepSeek API key")
    model: str = "deepseek-chat"
    base_url: str = "https://api.deepseek.com/v1/chat/completions"

    def _call(
        self,
        prompt: str,
        stop: List[str] = None,
        run_manager: CallbackManagerForLLMRun = None,
        **kwargs: Any,
    ) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 2048,
            "temperature": 0.1
        }

        try:
            response = requests.post(self.base_url, headers=headers, json=data)
            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            return f"Error calling DeepSeek API: {str(e)}"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

@dataclass
class Monument:
    name: str
    lat: float
    lon: float
    outdoor: bool
    hours_needed: int
    day_text: str
    venue_open: int
    venue_closed: int
    day_raw: List[int]  # Crowd levels by hour (0-23)
    temperatures: List[float]  # Temperatures by hour (0-23)
    date: str

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Monument':
        return cls(
            name=data['name'],
            lat=data['lat'],
            lon=data.get('long', data.get('lon', 0)),  # Handle both 'lon' and 'long'
            outdoor=data['outdoor'],
            hours_needed=data['hours_needed'],
            day_text=data['day_text'],
            venue_open=data['venue_open'],
            venue_closed=data['venue_closed'],
            day_raw=data['day_raw'],
            temperatures=data['temperatures'],
            date=data['date']
        )

from langchain.tools import BaseTool
from pydantic import Field

class ItineraryOptimizerTool(BaseTool):
    name: str = "itinerary_optimizer"
    description: str = "Optimizes tourist itinerary based on opening hours, weather, and crowd levels"

    def _run(self, monuments_json: str) -> str:
        try:
            monuments_data = json.loads(monuments_json)
            monuments = [Monument.from_dict(data) for data in monuments_data]
            optimized_itinerary = self.optimize_itinerary(monuments)
            return json.dumps(optimized_itinerary, indent=2)
        except Exception as e:
            return f"Error optimizing itinerary: {str(e)}"

    def _arun(self, monuments_json: str) -> str:
        return self._run(monuments_json)

    def optimize_itinerary(self, monuments: List[Monument]) -> List[Dict[str, Any]]:
        """
        Optimize the itinerary considering all constraints
        """
        # Score each monument for each possible start time
        scored_slots = []

        for monument in monuments:
            best_slots = self.find_best_time_slots(monument)
            for slot in best_slots:
                scored_slots.append({
                    'monument': monument,
                    'start_hour': slot['start_hour'],
                    'end_hour': slot['end_hour'],
                    'score': slot['score'],
                    'reasons': slot['reasons']
                })

        # Sort by score (higher is better)
        scored_slots.sort(key=lambda x: x['score'], reverse=True)

        # Greedy assignment to avoid conflicts
        scheduled_monuments = []
        occupied_hours = set()

        for slot in scored_slots:
            monument = slot['monument']
            start_hour = slot['start_hour']
            end_hour = slot['end_hour']

            # Check if this monument is already scheduled
            if any(s['name'] == monument.name for s in scheduled_monuments):
                continue

            # Check for time conflicts
            slot_hours = set(range(start_hour, end_hour))
            if not slot_hours.intersection(occupied_hours):
                # No conflict, schedule this monument
                scheduled_monuments.append({
                    'name': monument.name,
                    'start_time': f"{start_hour:02d}:00",
                    'end_time': f"{end_hour:02d}:00",
                    'duration_hours': monument.hours_needed,
                    'outdoor': monument.outdoor,
                    'score': slot['score'],
                    'optimization_reasons': slot['reasons']
                })
                occupied_hours.update(slot_hours)

        # Sort final itinerary by start time
        scheduled_monuments.sort(key=lambda x: x['start_time'])

        return scheduled_monuments

    def find_best_time_slots(self, monument: Monument) -> List[Dict[str, Any]]:
        """
        Find the best time slots for a monument considering all constraints
        """
        possible_slots = []

        # Generate all possible time slots
        for start_hour in range(24):
            end_hour = start_hour + monument.hours_needed

            # Check if slot fits within operating hours
            if not self.is_within_operating_hours(start_hour, end_hour, monument):
                continue

            # Calculate score for this time slot
            score, reasons = self.calculate_slot_score(start_hour, end_hour, monument)

            possible_slots.append({
                'start_hour': start_hour,
                'end_hour': end_hour,
                'score': score,
                'reasons': reasons
            })

        # Return top 3 best slots
        possible_slots.sort(key=lambda x: x['score'], reverse=True)
        return possible_slots[:3]

    def is_within_operating_hours(self, start_hour: int, end_hour: int, monument: Monument) -> bool:
        """
        Check if the time slot is within the monument's operating hours
        """
        venue_open = monument.venue_open
        venue_closed = monument.venue_closed

        # Handle venues that close past midnight (e.g., open 9, close 1 = 9am to 1am next day)
        if venue_closed < venue_open:
            # Venue is open past midnight
            if start_hour >= venue_open or end_hour <= venue_closed:
                return True
            # Also check if the entire slot is before midnight on the same day
            if start_hour >= venue_open and end_hour <= 24:
                return True
            return False
        else:
            # Normal operating hours (e.g., 9am to 6pm)
            return start_hour >= venue_open and end_hour <= venue_closed

    def calculate_slot_score(self, start_hour: int, end_hour: int, monument: Monument) -> Tuple[float, List[str]]:
        """
        Calculate a score for a time slot based on various factors
        """
        score = 100  # Base score
        reasons = []

        # Factor 1: Crowd levels (lower crowds = higher score)
        avg_crowd = np.mean([monument.day_raw[h % 24] for h in range(start_hour, end_hour)])
        crowd_penalty = avg_crowd * 0.5  # Penalty increases with crowd level
        score -= crowd_penalty

        if avg_crowd < 20:
            reasons.append(f"Low crowds (avg: {avg_crowd:.1f})")
        elif avg_crowd > 60:
            reasons.append(f"High crowds (avg: {avg_crowd:.1f}) - penalty applied")

        # Factor 2: Temperature for outdoor sites
        if monument.outdoor:
            avg_temp = np.mean([monument.temperatures[h % 24] for h in range(start_hour, end_hour)])

            # Optimal temperature range: 20-28°C
            if avg_temp < 18 or avg_temp > 35:
                temp_penalty = abs(avg_temp - 26) * 2  # Heavy penalty for extreme temps
                score -= temp_penalty
                reasons.append(f"Extreme temperature (avg: {avg_temp:.1f}°C) - penalty applied")
            elif 20 <= avg_temp <= 28:
                score += 10  # Bonus for optimal temperature
                reasons.append(f"Optimal temperature (avg: {avg_temp:.1f}°C)")
            else:
                reasons.append(f"Moderate temperature (avg: {avg_temp:.1f}°C)")

        # Factor 3: Time of day preferences
        if start_hour >= 8 and start_hour <= 10:
            score += 15  # Morning bonus
            reasons.append("Good morning start time")
        elif start_hour >= 14 and start_hour <= 16:
            score += 10  # Afternoon bonus
            reasons.append("Good afternoon time")
        elif start_hour >= 20 or start_hour <= 6:
            score -= 20  # Late night/early morning penalty
            reasons.append("Unusual hours - penalty applied")

        # Factor 4: Duration efficiency
        if monument.hours_needed <= 2:
            score += 5  # Bonus for quick visits
            reasons.append("Quick visit - scheduling flexibility")
        elif monument.hours_needed >= 4:
            if start_hour >= 9 and end_hour <= 17:
                score += 5  # Bonus for long visits during business hours
                reasons.append("Long visit scheduled during optimal hours")

        return max(score, 0), reasons

class TourismItineraryAgent:
    def __init__(self, deepseek_api_key: str):
        self.llm = DeepSeekLLM(api_key="sk-your_deepseek_api_key")
        self.tools = [ItineraryOptimizerTool()]
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )

    def optimize_itinerary(self, monuments_data: List[Dict[str, Any]]) -> str:
        """
        Main method to optimize an itinerary given a list of monuments
        """
        monuments_json = json.dumps(monuments_data)

        prompt = f"""
        You are a tourism itinerary optimization expert. Given the following tourist sites data,
        create an optimized daily itinerary that:

        1. Respects opening and closing times of each site
        2. Allocates the required hours_needed for each site
        3. Avoids outdoor sites during extreme temperatures
        4. Minimizes visits during peak crowd times (high day_raw values)
        5. Creates a logical flow for the day

        Use the itinerary_optimizer tool with this data:
        {monuments_json}

        After getting the optimized itinerary, provide additional insights about:
        - Why certain time slots were chosen
        - Any potential issues or considerations
        - Tips for the tourist based on the weather and crowd patterns
        """

        return self.agent.run(prompt)

In [ ]:
example_monuments = [
    {
        "name": "Cairo Tower",
        "lat": 30.0459751,
        "lon": 31.2242988,
        "outdoor": False,
        "hours_needed": 3,
        "day_text": "Monday",
        "venue_open": 9,
        "venue_closed": 1,  # 1 AM next day
        "day_raw": [0, 0, 0, 20, 25, 30, 35, 35, 40, 45, 55, 65, 75, 80, 80, 70, 60, 45, 25, 0, 0, 0, 0, 0],
        "temperatures": [26.3, 25.7, 25.1, 24.5, 23.9, 23.5, 23.2, 23.3, 24.3, 26.1, 27.7, 29.4, 30.9, 32.1, 33.0, 33.6, 34.2, 34.0, 33.8, 33.2, 32.0, 30.2, 28.5, 27.4],
        "date": "2025-08-04"
    },
    {
        "name": "Pyramids of Giza",
        "lat": 29.9792,
        "lon": 31.1342,
        "outdoor": True,
        "hours_needed": 4,
        "day_text": "Monday",
        "venue_open": 8,
        "venue_closed": 17,
        "day_raw": [0, 0, 5, 15, 25, 40, 60, 80, 90, 95, 90, 85, 80, 75, 70, 60, 45, 30, 15, 5, 0, 0, 0, 0],
        "temperatures": [26.3, 25.7, 25.1, 24.5, 23.9, 23.5, 23.2, 23.3, 24.3, 26.1, 27.7, 29.4, 30.9, 32.1, 33.0, 33.6, 34.2, 34.0, 33.8, 33.2, 32.0, 30.2, 28.5, 27.4],
        "date": "2025-08-04"
    }
]

# Replace with your actual DeepSeek API key
API_KEY = "sk-your_deepseek_api_key"

# Initialize the agent
agent = TourismItineraryAgent(API_KEY)

# Optimize the itinerary
result = agent.optimize_itinerary(example_monuments)
print("Optimized Itinerary:")
print(result)

/tmp/ipython-input-986025352.py:246: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  self.agent = initialize_agent(
/tmp/ipython-input-986025352.py:279: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.agent.run(prompt)




> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Thought: To create an optimized itinerary, I need to consider the opening hours, required time at each site, outdoor conditions, crowd levels, and logical flow. The Pyramids of Giza are outdoor and should be visited during cooler hours with lower crowds, while Cairo Tower is indoor and can be scheduled later in the day when temperatures are higher. I'll use the itinerary_optimizer tool to generate the best schedule.

Action: itinerary_optimizer  
Action Input:  
```json
{
  "sites": [
    {
      "name": "Cairo Tower",
      "lat": 30.0459751,
      "lon": 31.2242988,
      "outdoor": false,
      "hours_needed": 3,
      "day_text": "Monday",
      "venue_open": 9,
      "venue_closed": 1,
      "day_raw": [0, 0, 0, 20, 25, 30, 35, 35, 40, 45, 55, 65, 75, 80, 80, 70, 60, 45, 25, 0, 0, 0, 0, 0],
      "temperatures": [26.3, 25.7, 25.1, 24.5, 23.9, 23.5, 23.2, 23.3, 24.3, 26